# Favorite

In [11]:

import dspy
from typing import Optional


def create_favorite_predictor_signature(rubric: Optional[str] = None):
    """Create a FavoritePredictor signature with rubric embedded in instructions."""
    
    instruction = "Predict whether an article will be marked as favorite based on its title."
    
    if rubric:
        instruction += f"\n\nUse the following taste rubric as guidance:\n{rubric}"
    
    return dspy.Signature(
        "title -> is_favorite: bool",
        instruction
    )


def create_favorite_predictor_with_reasoning_signature(rubric: Optional[str] = None):
    """Create a FavoritePredictorWithReasoning signature with rubric embedded in instructions."""
    
    instruction = "Predict whether an article will be marked as favorite based on its title with detailed reasoning."
    
    if rubric:
        instruction += f"\n\nUse the following taste rubric as guidance:\n{rubric}"
    
    return dspy.Signature(
        "title -> is_favorite: bool, reasoning: str",
        instruction
    )


class TastePredictionModule(dspy.Module):
    """DSPy module for predicting favorite articles from titles."""
    
    def __init__(self, use_reasoning: bool = True, rubric: Optional[str] = None):
        super().__init__()
        self.use_reasoning = use_reasoning
        
        if use_reasoning:
            signature = create_favorite_predictor_with_reasoning_signature(rubric)
            self.predict_favorite = dspy.ChainOfThought(signature)
        else:
            signature = create_favorite_predictor_signature(rubric)
            self.predict_favorite = dspy.Predict(signature)
    
    def forward(self, title: str):
        """Predict if an article title indicates a favorite."""
        result = self.predict_favorite(title=title)
        return result

# Load dataset

In [15]:
import json
dataset_path = "/Users/edmar/Code/research/taste/main/data/reader_shortlist/test/dspy_examples.json"

with open(dataset_path, "r") as f:
    dataset = json.load(f)

# Convert to DSPy format
label_field = 'is_favorite' if 'is_favorite' in dataset[0] else 'is_shortlist'
test_examples = [
    dspy.Example(title=item['title'], is_favorite=item[label_field]).with_inputs('title')
    for item in dataset
]


# Run Evaluation

In [16]:
def accuracy_metric(example, pred, trace=None):
    """Simple accuracy metric for DSPy evaluation"""
    return example.is_favorite == pred.is_favorite

In [23]:
import dspy
from dspy.evaluate import Evaluate


evaluator = Evaluate(devset=test_examples,metric=accuracy_metric, num_threads=8, display_progress=True)

module = TastePredictionModule(use_reasoning=True, rubric="")

evaluator(module)

2025/07/06 13:08:01 ERROR dspy.utils.parallelizer: Error for Example({'title': "When Fine-Tuning Actually Makes Sense: A Developer's Guide", 'is_favorite': False}) (input_keys={'title'}): No LM is loaded.. Set `provide_traceback=True` for traceback.
2025/07/06 13:08:01 ERROR dspy.utils.parallelizer: Error for Example({'title': 'The Generalize Step In TDD', 'is_favorite': True}) (input_keys={'title'}): No LM is loaded.. Set `provide_traceback=True` for traceback.
2025/07/06 13:08:01 ERROR dspy.utils.parallelizer: Error for Example({'title': 'Human coders are still better than LLMs', 'is_favorite': True}) (input_keys={'title'}): No LM is loaded.. Set `provide_traceback=True` for traceback.
2025/07/06 13:08:01 ERROR dspy.utils.parallelizer: Error for Example({'title': 'Agencymaxxing', 'is_favorite': False}) (input_keys={'title'}): No LM is loaded.. Set `provide_traceback=True` for traceback.


  0%|          | 0/134 [00:00<?, ?it/s]

2025/07/06 13:08:01 ERROR dspy.utils.parallelizer: Error for Example({'title': 'More Lore of the World', 'is_favorite': False}) (input_keys={'title'}): No LM is loaded.. Set `provide_traceback=True` for traceback.


  0%|          | 0/134 [00:00<?, ?it/s]

2025/07/06 13:08:01 WARNING dspy.utils.parallelizer: Execution cancelled due to errors or interruption.


Exception: Execution cancelled due to errors or interruption.